# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компаниию, разработав такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Описание данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Напишем-функцию-для-шифровки/дешифровки-данных-датафрейма." data-toc-modified-id="Напишем-функцию-для-шифровки/дешифровки-данных-датафрейма.-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Напишем функцию для шифровки/дешифровки данных датафрейма.</a></span></li><li><span><a href="#Напишем-функцию-для-сравнения-предсказаний-открытых-данных-и-зашифрованных-." data-toc-modified-id="Напишем-функцию-для-сравнения-предсказаний-открытых-данных-и-зашифрованных-.-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Напишем функцию для сравнения предсказаний открытых данных и зашифрованных .</a></span></li></ul></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li></ul></div>

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except FileNotFoundError:
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(df.head(), df.tail())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В таблице данных компании соблюден почти идеальный баланс между мужчинами и женщинами. Средний возраст застрахованного составляет около 31 года, при этом медиана составляет 30. Границы возраста застрахованных приводят к выводу, что компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65 лет. Также интересно взглянуть на уровень заработной платы застрахованных - медиана составляет чуть более 40 000 рублей. В целом, в компании скорее представлен средний экономический класс, чем слишком бедные или слишком богатые граждане.

In [6]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [7]:
df.duplicated().sum()

153

Нет однозначного идентификатора клиента типа id или ФИО, поэтому мы не можем утверждать, что все совпадения являются дубликатами

### Описание данных

* Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Данные не имеют пропусков. В них присутствуют строки-дубликаты (153).
Также видим целочисленные признаки, таким как возраст и зарплата, имеющие тип данных "float".

Предобработка данных выходит за рамки проектной работы, однако приведем целочисленные знамения признаков к типу "int".


In [8]:
df[df.columns] = df[df.columns].astype('int')
df.dtypes

Пол                  int32
Возраст              int32
Зарплата             int32
Члены семьи          int32
Страховые выплаты    int32
dtype: object

## Умножение матриц

Признаки представляют собой матрицу размера 5000 х 4. Ключ – квадратная матрица размера 4 x 4. Для получения зашифрованных данных ключ матрица признаков умножается на ключ.

Для дальнейшей расшифровки данных необходимо, чтобы матрица-ключ была обратима.

В общем виде докажем, что качество линейной регрессии при умножении признаков на квадратную обратимую матрицу не изменится.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки (ключ)

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)


Предсказания:

$$
a = Xw \label{eq2}\tag{1}
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)  \\\tag{2}
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y  \\\tag{3}
$$

$$
P P^{-1} = P^{-1}P = E  \\\tag{4}
$$

$$
E X = X E = X  \\\tag{5}
$$


$$
(XP)^T = P^T X^T \\\tag{6}
$$

$$
(XP)^{-1} = P^{-1} X^{-1}  \\\tag{7}
$$


Произведем умножение матрицы признаков $X$ на матрицу-ключ $P$, рассчитаем новые веса ${w_1}$ по формуле (3):

$$
w_1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Тогда по свойствам (6) и (7):

$$
w_1 = (P^TX^TXP)^{-1} P^TX^T  y = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T  y
$$

По свойствам (4) и (5) имеем:

$$
w_1 = P^{-1}(X^TX)^{-1} E X^T  y = P^{-1}(X^TX)^{-1} X^T  y
$$

Получили вектор весов из формулы (3), подставим его:

$$
w_1 =  P^{-1}w
$$

Вычислим предсказания $a_1$:

$$
a_1 =  XPw_1 = XPP^{-1}w
$$

По свойству ассоциативности умножения, а также по свойствам (4) и (5) имеем:

$$
a_1 = XPP^{-1}w = XEw = Xw 
$$

$$
a_1 = a - ч.т.д.
$$

**Вывод:** Качество линейной регрессии при умножении матрицы признаков на обратимую матрицу не изменится.
Предсказания $a_1$ для матрицы признаков $X$, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.

## Алгоритм преобразования

**Алгоритм**

#### Напишем функцию для шифровки/дешифровки данных датафрейма.

In [9]:
def encryption(df, decoding=False):
    np.random.seed(3)
    P = np.random.randint(0, 100, (df.shape[1], df.shape[1]))
    df_array = np.array(df)
    if decoding: 
        P = np.linalg.inv(P)      
    return pd.DataFrame(data=(df_array @ P), columns=df.columns).round(0).astype(int)

Применим функцию к персональным данным клиентов. Проведем шифровку и дешифровку, чтобы убедиться, что функция работает корректно.

In [10]:
df_encrypt = encryption(df, decoding=False)

In [11]:
df_encrypt

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1042578,1885621,4764704,993779,2182891
1,799149,1444935,3651484,761914,1672604
2,441609,798551,2018146,421189,924290
3,876327,1585077,4004782,834913,1835172
4,548712,992335,2507728,523220,1148680
...,...,...,...,...,...
4995,750474,1357210,3429300,715200,1571242
4996,1101207,1991885,5032930,1049420,2306021
4997,712506,1288658,3255908,678872,1491962
4998,687465,1243138,3140926,655052,1439263


In [12]:
df_encrypt = encryption(df_encrypt, decoding=True)

In [13]:
display(df_encrypt, df)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4995,0,28,35700,2,0
4996,0,34,52400,1,0
4997,0,20,33900,2,0
4998,1,22,32700,3,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4995,0,28,35700,2,0
4996,0,34,52400,1,0
4997,0,20,33900,2,0
4998,1,22,32700,3,0


In [14]:
print(
    [(df_encrypt-df)[i].unique() for i in (df_encrypt-df)[df.columns]]
)

[array([0]), array([0]), array([0]), array([0]), array([0])]


Функция шифровки/дешифровки корректно преобразует данные.

Далее проверим, есть ли отличия в предсказаниях зашифрованный данных и открытых.

#### Напишем функцию для сравнения предсказаний открытых данных и зашифрованных .

In [15]:
def compare(X, X_encrypt, y):
    X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
    X_encrypt = np.concatenate((np.ones((X_encrypt.shape[0], 1)), X_encrypt), axis=1)
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    w_1 = np.linalg.inv(X_encrypt.T @ X_encrypt) @ X_encrypt.T @ y
    a = X @ w
    a_1 = X_encrypt @ w_1
    return [(a_1 - a).round(3) for i in range(len(a)) if abs(a[i].round(3) - a_1[i].round(3)) > 0]

In [16]:
X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']
X_encrypt = encryption(X, decoding=False)

In [17]:
compare(X, X_encrypt, y)

[]

Зашифрованные и открытые данные согласно функции наименьших  потерь дают одни и те же предсказания.

## Проверка алгоритма

In [18]:
model = LinearRegression()

In [19]:
model.fit(X, y)
predictions = model.predict(X)
print(f'''R2 исходных данных {r2_score(y, predictions)},
MSE исходных данных {mean_squared_error(y, predictions)}''')

R2 исходных данных 0.42494550308169177,
MSE исходных данных 0.12334688937098945


In [20]:
model.fit(X_encrypt, y)
predictions = model.predict(X_encrypt)
print(f'''R2 исходных данных {r2_score(y, predictions)},
MSE исходных данных {mean_squared_error(y, predictions)}''')

R2 исходных данных 0.4249455030816899,
MSE исходных данных 0.12334688937098985


### Вывод:

Создан простой алгоритм шифровки/дешифровки данных, таким образом в случае домножения матрицы признаков на случайную обратимую матрицу можно обеспечить защиту данных, не особо потеряв в качестве модели. (Незначительное отличие вызвано особенностью преобразования матриц и чисел с плавающей точкой)